# Adaptive PDE discretizations on cartesian grids 
## Volume : GPU accelerated methods
## Part : Reproducibility
## Chapter : Rander metrics

In this notebook, we solve Rander eikonal equations on the CPU and the GPU, and check that they produce consistent results.

*Note on the numerical schemes*: The numerical schemes solved by the CPU and GPU in the Rander case are entirely different. The CPU version uses a causal semi-Lagrangian scheme, two dimensional only, whereas the GPU version uses a non-causal (yet monotone) Eulerian scheme, in two and three dimensions.

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Isotropic_Repro','GPU'))

In [2]:
import cupy as cp
import numpy as np
import itertools
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

In [3]:
from agd import HFMUtils
from agd import AutomaticDifferentiation as ad
from agd import Metrics
from agd import FiniteDifferences as fd
from agd import LinearParallel as lp
import agd.AutomaticDifferentiation.cupy_generic as cugen

norm_infinity = ad.Optimization.norm_infinity
from agd.HFMUtils import RunGPU,RunSmart

In [4]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global HFMUtils,ad,cugen,RunGPU,RunSmart,Metrics
    HFMUtils,ad,cugen,RunGPU,Metrics = rreload([HFMUtils,ad,cugen,RunGPU,Metrics],"../..")    
    RunSmart = cugen.cupy_get_args(HFMUtils.RunSmart,dtype64=True,iterables=(dict,Metrics.Base))

In [5]:
cp = ad.functional.decorate_module_functions(cp,cugen.set_output_dtype32) # Use float32 and int32 types in place of float64 and int64
plt = ad.functional.decorate_module_functions(plt,cugen.cupy_get_args)
RunSmart = cugen.cupy_get_args(RunSmart,dtype64=True,iterables=(dict,Metrics.Base))

### 0.1 Utilities

In [6]:
#from Notebooks_GPU.ExportedCode.Isotropic_Repro import RunCompare
def RunCompare(gpuIn,check=True):
    gpuOut = RunGPU(gpuIn)
    if gpuIn.get('verbosity',1): print("---")
    cpuIn = gpuIn.copy(); cpuIn.pop('traits',None)
    cpuOut = RunSmart(cpuIn)
    print("Max |gpuValues-cpuValues| : ", norm_infinity(gpuOut['values'].get()-cpuOut['values']))
    cpuTime = cpuOut['FMCPUTime']; gpuTime = gpuOut['solverGPUTime'];
    print(f"Solver time (s). GPU : {gpuTime}, CPU : {cpuTime}. Device acceleration : {cpuTime/gpuTime}")
    assert not check or cp.allclose(gpuOut['values'],cpuOut['values'],atol=1e-5,rtol=1e-4)
    return gpuOut,cpuOut

In [7]:
factor_variants = [
    {}, # Default
    {"seedRadius":2}, # Spread seed information
    {"factorizationRadius":10,'factorizationPointChoice':'Key'} # Source factorization
]
multip_variants = [
    {'multiprecision':False}, # Default
    {'multiprecision':True}, # Reduces roundoff errors
]
order_variants = [
    {'order':1}, # Default
    {'order':2}, # More accurate on smooth instances
]

## 1. Two dimensions

### 1.1 Constant metric

In [40]:
ReloadPackages()

In [41]:
n=7
hfmIn = HFMUtils.dictIn({
    'model':'Rander2',
    'seeds':cp.array([[0.,0.]]),
    'exportValues':1,
    'nitermax_o':1,
    'raiseOnNonConvergence':False,
    'traits':{
        'niter_i':16,'shape_i':(8,8), # Best
    }
})
hfmIn.SetRect([[-1,1],[-1,1]],dimx=n+1)
hfmIn['metric'] = Metrics.Rander(cp.eye(2),cp.array([0,0.5]) )

Casting output of function array from float64 to float32
Casting output of function eye from float64 to float32
Casting output of function array from float64 to float32


In [42]:
metric=hfmIn['metric']
metric.dual().w

array([   -0, -0.667], dtype=float32)

In [43]:
gpuOut = RunGPU(hfmIn)

Setting the kernel traits.
Prepating the domain data (shape,metric,...)
Preparing the values array (setting seeds,...)
Preparing the GPU kernel
Setup and run the eikonal solver
Setting solver drift
(2,)
0.03125
[   16     0    16]
[    0 0.125]
['values', 'geom', 'drift', 'seedTags']
---- Warning ----
 Solver AGSI did not reach convergence after maximum allowed number 1 of iterations 
-----------------

GPU solve took 0.0015034675598144531 seconds, in 1 iterations.
Post-Processing


In [39]:
gpuOut['values']

array([[  2.9,   inf,   2.9,   inf,   2.9,   inf,   2.9,   inf],
       [  inf,   2.9,   inf,   2.9,   inf,   2.9,   inf,   2.9],
       [  2.9,   inf,   2.9,   inf,   2.9,   inf,   2.9,   inf],
       [  inf,   2.9,   inf,   2.9,   inf,   2.9,   inf,   2.9],
       [  2.9,   inf,   2.9,   inf,   2.9,   inf,   2.9,   inf],
       [  inf,   2.9,   inf,   2.9,   inf,   2.9,   inf,   2.9],
       [  2.9,   inf,   2.9,   inf,   2.9,   inf,   2.9,   inf],
       [  inf,   2.9,   inf,   2.9,   inf,   2.9,   inf,   2.9]], dtype=float32)

## 1.2 Zermelo problem

In [81]:
ReloadPackages()

In [82]:
n=10
hfmIn = HFMUtils.dictIn({
    'model':'Rander2',
    'seeds':cp.array([[0.,0.]]),
    'exportValues':1,
})
hfmIn.SetRect([[-2.*np.pi,2.*np.pi],[-2.*np.pi,2.*np.pi]],dimx=n+1)

Casting output of function array from float64 to float32


In [83]:
def Drift(x):
    mult = 0.5*np.sin(x[0])*np.sin(x[1]) / np.linalg.norm(x,axis=0)
    mult[np.isnan(mult)]=0.
    return mult*x

In [84]:
hfmIn.Grid().dtype

dtype('float32')

In [85]:
hfmIn['metric']=Metrics.Rander.from_Zermelo(cp.eye(2),Drift(hfmIn.Grid()))

Casting output of function eye from float64 to float32


In [86]:
RunGPU(hfmIn)

Setting the kernel traits.
Prepating the domain data (shape,metric,...)
Preparing the values array (setting seeds,...)
Preparing the GPU kernel
Setup and run the eikonal solver
(2, 11, 11)
0.71486384
[0.742 -0.00801 0.742]
[0.121 0.121]
GPU solve took 0.001999378204345703 seconds, in 1 iterations.
Post-Processing


{'keys': {'used': ['exportValues',
   'origin',
   'arrayOrdering',
   'model',
   'dims',
   'gridScale',
   'metric',
   'seeds'],
  'defaulted': OrderedDict([('verbosity', 1),
               ('help', []),
               ('traits',
                {'Scalar': numpy.float32,
                 'Int': numpy.int32,
                 'multiprecision_macro': 0,
                 'pruning_macro': 0,
                 'shape_i': (24, 24),
                 'niter_i': 48,
                 'ndim_macro': 2}),
               ('multiprecision', False),
               ('values_float64', False),
               ('factoringRadius', 0),
               ('order', 1),
               ('periodic', (False, False)),
               ('drift', None),
               ('dualMetric', None),
               ('overwriteMetric', False),
               ('tol', 0.0002513274116526427),
               ('seedValues', array([    0], dtype=float32)),
               ('seedRadius', 0.0),
               ('dummy_kernel', False),
      

In [9]:
help(Metrics.Rander)

Help on class Rander in module agd.Metrics.rander:

class Rander(agd.Metrics.base.Base)
 |  Rander(m, w)
 |  
 |  A Rander norm takes the form F(x) = sqrt(<x,m.x>) + <w,x>.
 |  Inputs : 
 |  - m : Symmetric positive definite matrix.
 |  - w : Vector, obeying <w,m^(-1).w> < 1
 |  
 |  Method resolution order:
 |      Rander
 |      agd.Metrics.base.Base
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, m, w)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |  
 |  anisotropy_bound(self)
 |      Upper bound on norm(u)/norm(v), 
 |      for any unit vectors u and v.
 |  
 |  dual(self)
 |      This function returns the dual 
 |      to a Rander norm, which turns out to have a similar algebraic form.
 |      The dual norm is defined as 
 |      F'(x) = sup{ <x,y>; F(y)<=1 }
 |  
 |  flatten(self)
 |  
 |  inv_transform(self, a)
 |      Affine transformation of the norm. 
 |      The new unit ball is the inverse ima